In [ ]:
import random

import ProductCatalogGetter
import schema
from schema import config, ignore_ncid_list, control_offer_ncid_list
from utils import upload_file_to_gcs, insertCSVIntoCloudSQL
from database import Database

def update_cache(data, context):
    '''
        Event-based Cloud Function that augments our existing product offering information
        in the OTAPI database with channel data & valid time periods from the TMF 620
        Product Catalog V1 API.
    '''
    print(f"data : {data}")
    print(f"context : {context}")
    prod_cat_getter = ProductCatalogGetter.ProductCatalogGetter(
        env = config["gcp"]["endpoint"],
        ignore_self_signed_certs = True,
        record_pull_size = 256
    )

    with Database(
        project_id                  = config["gcp"]["project_id"],
        instance_id                 = config["database"]["instance_id"],
        db_host                     = config["database"]["ip"],
        db_name                     = config["database"]["name"],
        db_port                     = config["database"]["port"],
        db_user                     = config["database"]["db_user"]["name"],
        db_user_pw_secret_id        = config["database"]["db_user"]["secret_id"],
        ssl_ind                     = config["database"]["ssl_ind"],
        ssl_server_ca_secret_id     = config["gcp"]["secrets"]["server_ca"],
        ssl_client_cert_secret_id   = config["gcp"]["secrets"]["client_cert"],
        ssl_client_key_secret_id    = config["gcp"]["secrets"]["client_key"]
    ) as pg_db:
        # Fetching list of NCIDs that we want to query
        results = pg_db.query("""
            SELECT distinct(trim(unnest(string_to_array(ncid_list, ',')))) as ncid
            FROM product_offering po;
        """)
        ncid_list = [
            row["ncid"] for row in results
            if row["ncid"] not in ignore_ncid_list + control_offer_ncid_list
        ]
        print(f'Number of distinct NCIDs that we want to query TMF 620 with: {len(ncid_list)}')
        if len(ncid_list) == 0:
            print("Number of fetched NCIDs are 0.")
            raise ValueError("Fetched 0 NCIDs from database, this should not happen.")

        # Kong access token
        print("authenticating..")
        prod_cat_getter.check_and_refresh_access_token()

        # Generate batch number
        batch_id = random.randint(1, 1000000000)
        # Fetching data from FIFA PC API, writing to local files
        print(f'Grabbing product offering data, writing to '
              f'{config["file_system"]["path"]}')
        product_offering_filters = schema.product_offering_filters
        product_offering_csv_columns = schema.product_offering_csv_columns
        prod_cat_getter.load_select_product_offers(
            ncid_list   = ncid_list,
            file_path   = config["file_system"]["path"],
            filters     = product_offering_filters,
            write_csv   = True,
            csv_columns = product_offering_csv_columns,
            batch_id    = batch_id
        )

        # Uploading local files into gcs
        product_offering_gcs_uri = upload_file_to_gcs(
            bucket_name     = config["gcp"]["bucket_name"],
            gcs_file_name   = config["gcp"]["file_name"],
            file_path       = config["file_system"]["path"]
        )

        # Importing gcs files into CloudSQL
        print(f'Importing product offering info into '
              f'{config["database"]["schema"]["t_pcpol"]["name"]}')
        insertCSVIntoCloudSQL(
            filePath        = product_offering_gcs_uri,
            projectId       = config["gcp"]["project_id"],
            instanceId      = config["database"]["instance_id"],
            databaseName    = config["database"]["name"],
            table           = config["database"]["schema"]["t_pcpol"]["name"],
            columns         = config["database"]["schema"]["t_pcpol"]["columns"]
        )

        print("Calling procedure process_product_catalog_loader_records in CloudSQL")
        pg_db.query(f'CALL process_product_catalog_loader_records({batch_id})')
    return "OK"

In [ ]:
import random

import ProductCatalogGetter
import schema
from schema import config, ignore_ncid_list, control_offer_ncid_list
from utils import upload_file_to_gcs, insertCSVIntoCloudSQL
from database import Database

prod_cat_getter = ProductCatalogGetter.ProductCatalogGetter(
    env = config["gcp"]["endpoint"],
    ignore_self_signed_certs = True,
    record_pull_size = 256
)



In [ ]:
import os 

os.environ